## Install Libraries

In [1]:
import os
import pandas as pd

import psycopg2

## Connect to the DB

In [2]:
# information used to create a database connection
sqluser = 'postgres'
dbname = 'mimic4'
hostname = 'localhost'
port_number = 5434
schema_name = 'omop_cdm'

# Connect to postgres with a copy of the MIMIC-III database
con = psycopg2.connect(dbname=dbname, user=sqluser, host=hostname, port=port_number, password='mysecretpassword')

# the below statement is prepended to queries to ensure they select from the right schema
query_schema = 'set search_path to ' + schema_name + ';'

## Create Schema

In [127]:
dropSchemaQuery = """drop schema if exists sepsis_icd cascade"""
createSchemaQuery = """create schema if not exists sepsis_icd"""

with con:
    with con.cursor() as cursor:
        cursor.execute(dropSchemaQuery)
        cursor.execute(createSchemaQuery)

## Cohort Selection

In [128]:
dropCohortPosQuery = """drop table if exists sepsis_icd.cohort cascade"""
cohortPosQuery = """
    create table sepsis_icd.cohort as
    select
    vis.visit_occurrence_id,
    min(vis.person_id) as person_id,
    1 as sepsis
    from
    omop_cdm.visit_occurrence vis
    inner join omop_cdm.person per
    on per.person_id = vis.person_id
    inner join mimiciv.patients pat
    on pat.subject_id = per.person_source_value::int
    inner join omop_cdm.condition_occurrence con
    on con.visit_occurrence_id = vis.visit_occurrence_id
    and con.condition_source_value in ('99591', '99592', '78552', 'A419', 'R6520', 'R6521')
    where visit_source_value NOT LIKE '%-%'
    and (floor(date_part('day', vis.visit_start_datetime - make_timestamp(pat.anchor_year, 1, 1, 0, 0, 0))/365.0) + pat.anchor_age) > 18
    group by vis.visit_occurrence_id
    having count(*) = 1
    ;
    """

with con:
    with con.cursor() as cursor:
        cursor.execute(dropCohortPosQuery)
        cursor.execute(cohortPosQuery)

In [129]:
cohortPosCountQuery = """select count(*) from sepsis_icd.cohort;"""
cohortPosCountDf = pd.read_sql_query(cohortPosCountQuery, con)
cohortPosCount = cohortPosCountDf['count'][0]
cohortPosCount

8885

In [131]:
cohortNegQuery = """
    insert into sepsis_icd.cohort
    with t1 as (
        select
        distinct
        vis.visit_occurrence_id,
        vis.person_id,
        0 as sepsis
        from
        omop_cdm.visit_occurrence vis
        inner join omop_cdm.person per
        on per.person_id = vis.person_id
        inner join mimiciv.patients pat
        on pat.subject_id = per.person_source_value::int
        inner join mimiciv.admissions adm
        on adm.subject_id = pat.subject_id
        inner join mimiciv.icustays icu
        on icu.hadm_id = adm.hadm_id
        where vis.visit_source_value not like '%-%'
        and (floor(date_part('day', vis.visit_start_datetime - make_timestamp(pat.anchor_year, 1, 1, 0, 0, 0))/365.0) + pat.anchor_age) > 18
        and vis.person_id not in (
            select
            con.person_id
            from
            omop_cdm.condition_occurrence con
            where
            con.condition_source_value in ('99591', '99592', '78552', 'A419', 'R6520', 'R6521')
        )
    )
    select * from t1
    order by random() limit 8885
    ;
    """

with con:
    with con.cursor() as cursor:
        cursor.execute(cohortNegQuery)

## Static Data

In [132]:
dropStaticQuery = """drop table if exists sepsis_icd.static cascade"""
staticQuery = """
    create table sepsis_icd.static as
    select
    vo.person_id AS person_id,
    vo.visit_start_datetime AS visit_start_datetime,
    vo.visit_end_datetime AS visit_end_datetime,
    vo.visit_occurrence_id AS visit_occurrence_id,
    con_vo.concept_name AS visit_occurrence_concept_name,
    (date_part('day', (vo.visit_end_datetime - vo.visit_start_datetime)) * 24) + date_part('hour', (vo.visit_end_datetime - vo.visit_start_datetime)) as visit_duration_hrs,
    con_src.concept_name AS visit_source_concept_name,
    vo.admitting_source_value AS admitting_source_value
    from
    omop_cdm.visit_occurrence vo
    inner join omop_cdm.concept con_vo
    ON con_vo.concept_id = vo.visit_concept_id
    inner join omop_cdm.concept con_src
    ON con_src.concept_id = vo.visit_source_concept_id
    inner join sepsis_icd.cohort coh
    on coh.visit_occurrence_id = vo.visit_occurrence_id and coh.person_id = vo.person_id
    ;
    """

with con:
    with con.cursor() as cursor:
        cursor.execute(dropStaticQuery)
        cursor.execute(staticQuery)

In [133]:
staticCountQuery = """select count(*) from sepsis_icd.static;"""
staticCountDf = pd.read_sql_query(staticCountQuery, con)
staticCount = staticCountDf['count'][0]
staticCount

17770

## Vitals Data

In [134]:
dropVitalsQuery = """drop table if exists sepsis_icd.vitals cascade"""
vitalsQuery = """
    create table sepsis_icd.vitals as
    with vitals_stg_1 as
    (
        select
        mmt.visit_occurrence_id as visit_occurrence_id,
        mmt.person_id as person_id,
        mmt.measurement_datetime as measurement_datetime,
        mmt.unit_source_value as unit_source_value,
        mmt.value_as_number as value_as_number,
        cpt.concept_name as concept_name
        from
        omop_cdm.measurement mmt
        inner join omop_cdm.concept cpt
        on cpt.concept_id = mmt.measurement_concept_id
        inner join sepsis_icd.cohort coh
        on coh.visit_occurrence_id = mmt.visit_occurrence_id and coh.person_id = mmt.person_id
        where
        measurement_concept_id in (
        3027018 -- Heart rate
        , 21492239, 3004249 -- Systolic blood pressure
        , 21492240, 3012888 -- Diastolic blood pressure
        , 3027598, 21492241 -- Mean blood pressure
        , 1175625, 3024171, 3007469 -- Respiratory rate
        , 3020891 -- Body temperature
        , 40762499 -- Oxygen saturation in Arterial blood by Pulse oximetry
        , 3016335 -- Glasgow coma score eye opening
        , 3009094 -- Glasgow coma score verbal
        , 3008223 -- Glasgow coma score motor
        )
        and value_as_number is not null
    )
    , vitals_stg_2 AS
    (
      select
        visit_occurrence_id,
        person_id,
        measurement_datetime,
        unit_source_value,
        value_as_number,
        concept_name,
        row_number() over (partition by person_id, concept_name order by measurement_datetime) as rn
      from vitals_stg_1
    )
    select * from vitals_stg_2
    """

with con:
    with con.cursor() as cursor:
        cursor.execute(dropVitalsQuery)
        cursor.execute(vitalsQuery)

In [135]:
vitalsCountQuery = """select count(*) from sepsis_icd.vitals;"""
vitalsCountDf = pd.read_sql_query(vitalsCountQuery, con)
vitalsCount = vitalsCountDf['count'][0]
vitalsCount

8554241

## Lab Measurements Data

In [136]:
dropLabsQuery = """drop table if exists sepsis_icd.labs cascade"""
labsQuery = """
    create table sepsis_icd.labs as
with labs_stg_1 as
    (
        select
        mmt.visit_occurrence_id as visit_occurrence_id,
        mmt.person_id AS person_id,
        measurement_datetime as measurement_datetime,
        unit_source_value as unit_source_value,
        value_as_number as value_as_number,
        cpt.concept_name as concept_name
        from
        etl_dataset_temp.measurement mmt
        inner join omop_cdm.concept cpt
        on cpt.concept_id = mmt.measurement_concept_id
        inner join sepsis_icd.cohort coh
        on coh.visit_occurrence_id = mmt.visit_occurrence_id and coh.person_id = mmt.person_id
        where
        measurement_concept_id in (
        3047181	-- Lactate [Moles/volume] in Blood
		, 3013290	-- Carbon dioxide [Partial pressure] in Blood
		, 3024561	-- Albumin [Mass/volume] in Serum or Plasma
		, 3024629	-- Glucose [Mass/volume] in Urine by Test strip
		, 3008939	-- Band form neutrophils [#/volume] in Blood by Manual count
		, 3012501	-- Base excess in Blood by calculation
		, 3005456	-- Potassium [Moles/volume] in Blood
		, 3010421	-- pH of Blood
		, 3014576	-- Chloride [Moles/volume] in Serum or Plasma
		, 3031147	-- Carbon dioxide, total [Moles/volume] in Blood by calculation
		, 3024128	-- Bilirubin.total [Mass/volume] in Serum or Plasma
		, 3000905	-- Leukocytes [#/volume] in Blood by Automated count
		, 3016723	-- Creatinine [Mass/volume] in Serum or Plasma
		, 3022217	-- INR in Platelet poor plasma by Coagulation assay
		, 3019550	-- Sodium [Moles/volume] in Serum or Plasma
		, 3000285	-- Sodium [Moles/volume] in Blood
		, 3000963	-- Hemoglobin [Mass/volume] in Blood
		, 3000963	-- Hemoglobin [Mass/volume] in Blood
		, 3018672	-- pH of Body fluid
		, 3024929	-- Platelets [#/volume] in Blood by Automated count
		, 3013682	-- Urea nitrogen [Mass/volume] in Serum or Plasma
		, 3004501	-- Glucose [Mass/volume] in Serum or Plasma
		, 3018572	-- Chloride [Moles/volume] in Blood
		, 3027315	-- Oxygen [Partial pressure] in Blood
		, 3016293	-- Bicarbonate [Moles/volume] in Serum or Plasma
		, 3023103	-- Potassium [Moles/volume] in Serum or Plasma
		, 3037278	-- Anion gap 4 in Serum or Plasma
		, 3003282	-- Leukocytes [#/volume] in Blood by Manual count
		, 3023314	-- Hematocrit [Volume Fraction] of Blood by Automated count
		, 3013466	-- aPTT in Blood by Coagulation assay
        )
        and value_as_number is not null
    )
    , labs_stg_2 as
    (
      select
        visit_occurrence_id,
        person_id,
        measurement_datetime,
        unit_source_value,
        value_as_number,
        concept_name,
        row_number() over (partition by person_id, concept_name order by measurement_datetime) as rn
      from labs_stg_1
    )
    select * from labs_stg_2
    """

with con:
    with con.cursor() as cursor:
        cursor.execute(dropLabsQuery)
        cursor.execute(labsQuery)

In [137]:
labsCountQuery = """select count(*) from sepsis_icd.labs;"""
labsCountDf = pd.read_sql_query(labsCountQuery, con)
labsCount = labsCountDf['count'][0]
labsCount

514102

## Mortality Data

In [138]:
dropMortalityQuery = """drop table if exists sepsis_icd.mortality cascade"""
mortalityQuery = """
    create table sepsis_icd.mortality as
    select
    distinct
    vo.visit_occurrence_id as visit_occurrence_id,
    vo.person_id as person_id,
    case when dth.person_id is null then false else (vo.visit_end_datetime = dth.death_datetime) end as discharge_mortality,
    case when dth.person_id is null then false else (vo.visit_end_datetime + interval '1 day' >= dth.death_datetime) end as one_day_mortality,
    case when dth.person_id is null then false else (vo.visit_end_datetime + interval '2 day' >= dth.death_datetime) end as two_day_mortality,
    case when dth.person_id is null then false else (vo.visit_end_datetime + interval '30 day' >= dth.death_datetime) end as thirty_day_mortality,
    case when dth.person_id is null then false else (vo.visit_end_datetime + interval '60 day' >= dth.death_datetime) end as sixty_day_mortality,
    case when dth.person_id is null then false else (vo.visit_end_datetime + interval '90 day' >= dth.death_datetime) end as ninety_day_mortality
    from
    omop_cdm.visit_occurrence vo
    inner join omop_cdm.person per
    on per.person_id = vo.person_id
    left join omop_cdm.death dth
    on dth.person_id = per.person_id
    inner join sepsis_icd.cohort coh
    on coh.visit_occurrence_id = vo.visit_occurrence_id and coh.person_id = vo.person_id
    ;
    """
with con:
    with con.cursor() as cursor:
        cursor.execute(dropMortalityQuery)
        cursor.execute(mortalityQuery)

In [139]:
mortalityCountQuery = """select count(*) from sepsis_icd.mortality;"""
mortalityCountDf = pd.read_sql_query(mortalityCountQuery, con)
mortalityCount = mortalityCountDf['count'][0]
mortalityCount

17770